# Grad-CAM Explainability

This notebook loads the trained EfficientNetB3 plant disease model and uses Grad-CAM to visualize **which regions of a plant image the model focuses on** when making a prediction.

You can:
- Pick a test image
- See prediction + Grad-CAM heatmap + overlay.
